In [3]:
#IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from scipy.spatial.transform import Rotation as R

import dataUtils as du

# LOAD DATA

In [4]:
path_base = "Tracking/Gridsearch/"
a = 100 #acceleration in steps
hz = 30
samples = 3 #steps in gridsearch
stepsize = 20 #steps used to perform gridsearch
full_path = "" #custom path to gridsearch data
num_meta_rows = 6

#load all data
df = du.load_dataset(a,samples,stepsize,path_base,full_path)

# Change rotations to accurate coordinate system and make translations relative to frame 0
df = du.fixCoordinates(df)

#get valid frames (frames without wobeling/ocscilation)
start_offset =  65 #(frame before action)
wanted_frames = samples ** 6 #amount of frames to extract
frame_offset = 2*hz #time between moves
spacing = 10 #how many previous frames that are considered when testing for oscillation
t_eps = 0.5 #translational error
r_eps =0.5 #rotational error

df_valid, valid_idx = du.getValidFrames(df,wanted_frames,frame_offset,start_offset,spacing,t_eps,r_eps)

#load targets
comb_list = du.getGridsearchCableLengths(stepsize)
comb_list = du.getValidTargets(comb_list,valid_idx)
Y = np.array(comb_list)

seed = 42

TOTAL FRAMES: 729 (729)
INVALID FRAMES: 0, VALID FRAMES: 729


# NORMALIZE DATA


In [5]:
#Regularize the input data
def raw(norm = True):
  col = ["X_trans", "Y_trans", "Z_trans", "X_rot", "Y_rot", "Z_rot"]
  X = df_valid[col].copy()
  if norm:
    for i in range(6):
      X[col[i]] = 2 * (X[col[i]] - X[col[i]].min()) / (X[col[i]].max() - X[col[i]].min()) - 1


  # Split data into train and test sets
  X_train, X_test, y_train, y_test = train_test_split(X.values, Y, test_size=0.15, random_state=seed)
  return X_train, X_test, y_train, y_test

In [6]:
# Marker normalize
def marker(norm = True):
  # min-max scale translations to be in [-1,1]
  col = [ "X_trans", "Y_trans", "Z_trans"]
  X_temp = df_valid.copy()
  for c in col:
    X_temp[c] = 2 * (X_temp[c] - X_temp[c].min()) / (X_temp[c].max() - X_temp[c].min()) - 1
  # Selecting features from column index 2 to 7 (exclusive of 8)
  X =X_temp.iloc[:, 2:8].values
  rotations = X[:, :3]
  translations = X[:, 3:]

  # Define the positions of 4 points spaced around the original pivot point
  # These positions are relative to the pivot point before rotation
  point_offsets = np.array([[1, 0, 1],  # Point 1
                            [0, 1, -1],  # Point 2
                            [-1, -1, 0], # Point 3
                            [-1, 0, -1]])# Point 4

  # Convert Euler angles to rotation matrices
  rot_matrices = R.from_euler('xyz', rotations, degrees=True).as_matrix()

  # Apply rotation to the point offsets
  rotated_points = np.matmul(rot_matrices, point_offsets.transpose())
  rotated_points = np.transpose(rotated_points, axes=(0, 2, 1))


  for i in range(rotated_points.shape[0]):
    for point in rotated_points[i]:
      point[0] += translations[i,0]
      point[1] += translations[i,1]
      point[2] += translations[i,2]

  X = rotated_points.reshape(-1,12)
  if norm:
    # 12 features, xyz xyz xyz xyz
    min_xs, max_xs = min([np.min(X[:,0]),np.min(X[:,3]),np.min(X[:,6]),np.min(X[:,9])]), max([np.max(X[:,0]),np.max(X[:,3]),np.max(X[:,6]),np.max(X[:,9])])
    min_ys, max_ys = min([np.min(X[:,1]),np.min(X[:,4]),np.min(X[:,7]),np.min(X[:,10])]), max([np.max(X[:,1]),np.max(X[:,4]),np.max(X[:,7]),np.max(X[:,10])])
    min_zs, max_zs = min([np.min(X[:,2]),np.min(X[:,5]),np.min(X[:,8]),np.min(X[:,11])]), max([np.max(X[:,2]),np.max(X[:,5]),np.max(X[:,8]),np.max(X[:,11])])

    #normalize all vectors based on their max/min x,y or z value
    for i in range(X.shape[0]):
      for x in [0,3,6,9]:
        X[i,x] = 2*((X[i,x]-min_xs)/(max_xs-min_xs)) - 1
      for y in [1,4,7,10]:
        X[i,y] = 2*((X[i,y]-min_ys)/(max_ys-min_ys)) -1
      for z in [2,5,8,11]:
        X[i,z] = 2*((X[i,z]-min_zs)/(max_zs-min_zs)) -1


  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=seed)
  return X_train, X_test, y_train, y_test

In [7]:
def marker_split(norm = True):
  # min-max scale translations to be in [-1,1]
  col = [ "X_trans", "Y_trans", "Z_trans"]
  X_norm = df_valid.copy()
  if norm:
    for c in col:
      X_norm[c] = 2 * (X_norm[c] - X_norm[c].min()) / (X_norm[c].max() - X_norm[c].min()) - 1
  # Selecting features from column index 2 to 7 (exclusive of 8)
  X =X_norm.iloc[:, 2:8].values
  rotations = X[:, :3]
  translations = X[:, 3:]
  # Define the positions of 4 points spaced around the original pivot point
  # These positions are relative to the pivot point before rotation
  point_offsets = np.array([[1, 0, 1],  # Point 1
                            [0, 1, -1],  # Point 2
                            [-1, -1, 0], # Point 3
                            [-1, 0, -1]])# Point 4

  # Convert Euler angles to rotation matrices
  rot_matrices = R.from_euler('xyz', rotations, degrees=True).as_matrix()

  # Apply rotation to the point offsets
  rotated_points = np.matmul(rot_matrices, point_offsets.transpose())
  rotated_points = np.transpose(rotated_points, axes=(0, 2, 1))
  rotated_points = rotated_points.reshape(-1,12)
  X =np.concatenate((rotated_points,translations),axis = 1) #makes translations/rotation be separate

  if norm:
  # vector normalize each "rotation" marker
    for i in range(X.shape[0]):
      for x in [0,3,6,9]:
        X[i,x] = X[i,x] / np.sqrt(2)
      for y in [1,4,7,10]:
        X[i,y] = X[i,y] / np.sqrt(2)
      for z in [2,5,8,11]:
        X[i,z] = X[i,z] / np.sqrt(2)

  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=seed)
  return X_train, X_test, y_train, y_test

In [8]:
X_train, X_test, y_train, y_test = raw(norm = True)

# LOAD MODEL VERSION

In [9]:
# Create the model
class FeedForwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = torch.load("Models/Final_models/2hid_32_neu_500epoch_p001lr_norm-11.pth")

# Define loss function and optimizer
criterion = nn.MSELoss()
L1 = nn.L1Loss()

batch_size = 1

# Setup data tensor
train_data_tensor = torch.tensor(X_train, dtype=torch.float32)
targets_tensor = torch.tensor(y_train, dtype=torch.float32)
val_data_tensor = torch.tensor(X_test, dtype=torch.float32)
val_targets_tensor = torch.tensor(y_test, dtype=torch.float32)

# Combine the input data and target values into a TensorDataset
train_dataset = TensorDataset(train_data_tensor, targets_tensor)
test_dataset = TensorDataset(val_data_tensor, val_targets_tensor)

# Create a DataLoader for batch processing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
def toNumpy(tensors):
  arr = [tensor.numpy() for tensor in tensors]
  arr = np.array(arr)
  return arr

# TEST MODEL ON DATA

In [11]:
#TRAINING
# Lists to store training and validation losses
val_pred = []
val_target = []
val_losses = []
val_l1 = []
train_pred = []
train_target = []
train_losses = []
train_l1 = []

# Initialize early stopping parameters
# best_val_loss = np.Inf
# patience = 15
# counter = 0

# Validation loop
model.eval()  # Set the model to evaluation mode
running_val_loss = 0.0
running_train_loss = 0.0
with torch.no_grad():
    for inputs, targets in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        running_train_loss += loss.item()
        train_losses.append(loss.item())
        l1 = L1(outputs,targets)
        train_l1.append(l1.item())
        train_pred.append(outputs)
        train_target.append(targets)
with torch.no_grad():
    for inputs, targets in val_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        running_val_loss += loss.item()
        val_losses.append(loss.item())
        l1 = L1(outputs,targets)
        val_l1.append(l1.item())
        val_pred.append(outputs)
        val_target.append(targets)
# Compute average validation loss for the epoch
average_val_loss = running_val_loss / len(val_loader)
average_train_loss = running_train_loss / len(train_loader)

# Print training and validation loss for the epoch
print(f"Mean Training MSE: {average_train_loss}, Mean Training STD: {np.std(np.array(train_losses))}")
print(f"Mean Training L1: {np.mean(np.array(train_l1))}, Mean Training STD: {np.std(np.array(train_l1))}")
print(f"Mean Test MSE: {average_val_loss}, Mean Test STD: {np.std(np.array(val_losses))}")
print(f"Mean Test L1: {np.mean(np.array(val_l1))}, Mean Test STD: {np.std(np.array(val_l1))}")

Mean Training MSE: 4.647031620168917, Mean Training STD: 8.214359908209905
Mean Training L1: 1.1769864114786774, Mean Training STD: 0.6323048179990226
Mean Test MSE: 7.238596547869119, Mean Test STD: 13.303230572219618
Mean Test L1: 1.3606037670915776, Mean Test STD: 0.8504878335140799


In [12]:
train_pred = toNumpy(train_pred)
train_target = toNumpy(train_target)
test_pred = toNumpy(val_pred)
test_target = toNumpy(val_target)

In [13]:
# Calculate MSE loss for each column
mse_train = np.mean((train_pred - train_target)**2, axis=0)
l1_train = np.mean((abs(train_pred - train_target)), axis=0)
mse_test = np.mean((test_pred - test_target)**2, axis=0)
l1_test = np.mean((abs(test_pred - test_target)), axis=0)
print(f"MSE Train: {mse_train[0]}")
print(f"L1 Train: {l1_train[0]}")
print(f"MSE Test: {mse_test[0]}")
print(f"L1 Test: {l1_test[0]}")

MSE Train: [ 1.9508692   0.6859009   1.9347792   0.5004356   0.58965886 22.220545  ]
L1 Train: [0.9910788 0.6613938 0.8443406 0.5564867 0.5927506 3.4158654]
MSE Test: [ 2.4328365   0.95128113  2.7785864   0.67148775  0.91180336 35.685577  ]
L1 Test: [1.1293923  0.8010302  0.8824812  0.63860023 0.70215005 4.0099688 ]
